Agda assignment - LDI 2019

Prove the lemmas given below by filling all holes `?` with your definitions.
It is not allowed to:

1. change/erase given definitions and lemma statements, section header/footer, variable A declaration, etc.,
2. introduce your own postulates, parameters, variables, hypotheses etc.,  
3. import modules other than what is already presented in this notebook.

It is allowed to:

1. introduce your own definitions and auxiliary lemmas,
2. change the order of lemmas to prove,
3. add comments.

Submit your solution via moodle before 23:59 on 14.06.2019. 
You should submit one file named Assignment-Shuffle.ipynb containing your proofs.

The author of the assignment and the grader is Lorenzo Clemente.

# Imports

## Logical connectives

````
module code.conn where

infixr 2 _∧_
data _∧_ (A : Set) (B : Set) : Set where
  ⟨_,_⟩ : A → B → A ∧ B

fst : {A B : Set} → A ∧ B → A
fst ⟨ a , _ ⟩ = a

snd : {A B : Set} → A ∧ B → B
snd ⟨ _ , b ⟩ = b

infixr 1 _∨_ 
data _∨_ (A : Set) (B : Set) : Set where
    left : A → A ∨ B
    right : B → A ∨ B
    
case : {A B C : Set} → A ∨ B → (A → C) → (B → C) → C
case (left a) f g = f a
case (right b) f g = g b

data ⊤ : Set where
  tt : ⊤
  
data ⊥ : Set where

⊥-elim : {A : Set} → ⊥ → A
⊥-elim ()

infix 3 ¬_ -- higher priority than ∨ and ∧

¬_ : Set → Set
¬ A = A → ⊥

infixr 1 _⟺_
_⟺_ : (A : Set) (B : Set) → Set
A ⟺ B = (A → B) ∧ (B → A)
````

## Natural numbers `N`

````
module code.nat where
open import code.conn public

data ℕ : Set where
  zero : ℕ
  suc : ℕ → ℕ

{-# BUILTIN NATURAL ℕ #-}

infixl 5 _+_
_+_ : ℕ → ℕ → ℕ
zero + m = m
(suc n) + m = suc (n + m)

infix 4 _≡ℕ_
data _≡ℕ_ : ℕ → ℕ → Set where
  z≡z : 0 ≡ℕ 0
  s≡s : {m n : ℕ} → m ≡ℕ n → suc m ≡ℕ suc n

postulate ≡ℕ-refl : {n : ℕ} → n ≡ℕ n
postulate ≡ℕ-trans : {m n p : ℕ} → m ≡ℕ n → n ≡ℕ p → m ≡ℕ p
postulate ≡ℕ-sym : {m n : ℕ} → m ≡ℕ n → n ≡ℕ m
postulate ≡ℕ-cong : {m n : ℕ} (f : ℕ → ℕ) → m ≡ℕ n → f m ≡ℕ f n
postulate suc-lemma : ∀ {m n} → suc (m + n) ≡ℕ m + suc n

infix  1 beginℕ_
infixr 2 _≡ℕ⟨⟩_ _≡ℕ⟨_⟩_
infix  3 _∎ℕ

beginℕ_ : {x y : ℕ} → x ≡ℕ y → x ≡ℕ y
beginℕ x≡y = x≡y

_≡ℕ⟨⟩_ : (x : ℕ) {y : ℕ} → x ≡ℕ y → x ≡ℕ y
x ≡ℕ⟨⟩ x≡y = x≡y

_≡ℕ⟨_⟩_ : (x : ℕ) {y z : ℕ} → x ≡ℕ y → y ≡ℕ z → x ≡ℕ z
x ≡ℕ⟨ x≡y ⟩ y≡z = ≡ℕ-trans x≡y y≡z

_∎ℕ : (x : ℕ) → x ≡ℕ x
x ∎ℕ = ≡ℕ-refl {x}

infix 4 _≤_
data _≤_ : ℕ → ℕ → Set where
  0≤n : {n : ℕ} → 0 ≤ n
  s≤s : {m n : ℕ} → m ≤ n → suc m ≤ suc n
````

## Lists

````
module code.list where
open import code.conn public
open import code.nat public

infixr 5 _∷_
data _* (A : Set) : Set where
  ε : A *
  _∷_ : A → A * → A *

infixr 5 _++_
_++_ : {A : Set} → A * → A * → A *
ε ++ ys = ys
(x ∷ xs) ++ ys = x ∷ (xs ++ ys)

length : {A : Set} → A * → ℕ
length ε = 0
length (_ ∷ xs) = suc (length xs)

reverse : {A : Set} → A * → A *
reverse ε = ε
reverse (a ∷ as) = reverse as ++ a ∷ ε

map : {A B : Set} → (A → B) → A * → B *
map _ ε = ε
map f (a ∷ as) = f a ∷ map f as

data 𝔹 : Set where
  true : 𝔹
  false : 𝔹

-- equality on booleans
data _≡𝔹_ : 𝔹 → 𝔹 → Set where
  tt : true ≡𝔹 true
  ff : false ≡𝔹 false

filter : {A : Set} → (A → 𝔹) → A * → A *
filter _ ε = ε
filter p (a ∷ as) with p a
... | true = a ∷ filter p as
... | false = filter p as

infix 4 _≡L_
data _≡L_ {A : Set} : A * → A * → Set where
  ε≡ε : ε ≡L ε
  ∷≡∷ : {a : A} {as bs : A *} → as ≡L bs → a ∷ as ≡L a ∷ bs

≡L-refl : {A : Set} {as : A *} → as ≡L as
≡L-refl {A} {ε} = ε≡ε
≡L-refl {A} {a ∷ as} = ∷≡∷ (≡L-refl {A} {as})

≡L-sym : {A : Set} {as bs : A *} → as ≡L bs → bs ≡L as
≡L-sym ε≡ε = ε≡ε
≡L-sym {A} (∷≡∷ {a} {as} {bs} as≡bs) = ∷≡∷ {A} {a} {bs} {as} (≡L-sym {A} {as} {bs} as≡bs)

≡L-trans : {A : Set} {as bs cs : A *} → as ≡L bs → bs ≡L cs → as ≡L cs
≡L-trans ε≡ε ε≡ε = ε≡ε
≡L-trans (∷≡∷ as≡bs) (∷≡∷ bs≡cs) = ∷≡∷ (≡L-trans as≡bs bs≡cs)

infix 4 _∈_
data _∈_ {A : Set} : A → A * → Set where
  stop : {a : A} {as : A *} → a ∈ a ∷ as
  skip : {a b : A} {as : A *} → a ∈ as → a ∈ b ∷ as

_ : 2 ∈ 0 ∷ 1 ∷ 2 ∷ 3 ∷ ε
_ = skip (skip stop)
````

# Assignment: Shuffle

````
module code.shuffle where
open import code.list public
````

We define the following notion of *shuffle*,.

````
data Shuffle {A : Set} : A * → A * → A * → Set where
    start : Shuffle ε ε ε
    left : ∀ {a as bs cs} → Shuffle as bs cs → Shuffle (a ∷ as) bs (a ∷ cs)
    right : ∀ {a as bs cs} → Shuffle as bs cs → Shuffle as (a ∷ bs) (a ∷ cs)
````

## **Exercise 1**

In [ ]:
{-# OPTIONS --allow-unsolved-metas #-} 
module code.assignment.ex1 where
open import code.shuffle public

empty : {A : Set} (as : A *) → Shuffle ε as as
empty = ?

?0 : (as : A *) → Shuffle ε as as


## **Exercise 2**

In [ ]:
module code.assignment.ex2 where
open import code.shuffle public

empty2 : {A : Set} (as bs cs : A *) → Shuffle as bs cs → as ≡L ε → bs ≡L cs
empty2 = ?

?0 : (as bs cs : A *) → Shuffle as bs cs → as ≡L ε → bs ≡L cs


## **Exercise 3**

In [93]:
{-# OPTIONS --allow-unsolved-metas #-} 
module code.assignment.ex3 where
open import code.shuffle public

sym : {A : Set} {as bs cs : A *} → Shuffle as bs cs → Shuffle bs as cs
sym = ?

?0 : Shuffle as bs cs → Shuffle bs as cs


## **Exercise 4**

*Hint:* It will probably be useful to have an auxiliary lemma first.

In [ ]:
module code.assignment.ex4 where
open import code.shuffle public
open import code.assignment.ex1 public
open import code.assignment.ex3 public

rev : {A : Set} (as bs cs : A *) →
    Shuffle as bs cs →
    Shuffle (reverse as) (reverse bs) (reverse cs)
rev = ?

?0
  : (as bs cs : A *) →
    Shuffle as bs cs → Shuffle (reverse as) (reverse bs) (reverse cs)


## **Exercise 5**

In [97]:
module code.assignment.ex5 where
open import code.shuffle public

preserve1 : {A : Set} {a : A} {as bs cs : A *} →
    Shuffle as bs cs →
    a ∈ cs →
    a ∈ as ∨ a ∈ bs
preserve1 = ?

?0 : Shuffle as bs cs → a ∈ cs → a ∈ as ∨ a ∈ bs


## **Exercise 6**

In [99]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.assignment.ex6 where
open import code.shuffle public

preserve2 : {A : Set} {a : A} {as bs cs : A *} →
    Shuffle as bs cs →
    a ∈ as ∨ a ∈ bs →
    a ∈ cs
preserve2 = ?

?0 : Shuffle as bs cs → a ∈ as ∨ a ∈ bs → a ∈ cs


## **Exercise 7**

````
module code.shuffle.in2 where
open import code.shuffle public
````

Consider the following variant of the membership predicate.

````
infix 4 _,_∈_
data _,_∈_ {A : Set} : A → A → A * → Set where
  stop : {a b : A} {as : A *} → b ∈ as → a , b ∈ a ∷ as
  skip : {a b c : A} {as : A *} → a , b ∈ as → a , b ∈ c ∷ as
````

Prove that `Shuffle` respects the *order* of occurrences of elements.

In [102]:
module code.assignment.ex7 where
open import code.shuffle.in2 public
open import code.assignment.ex6 public

order : {A : Set} {a b : A} {as bs cs : A *} →
    Shuffle as bs cs →
    a , b ∈ as →
    a , b ∈ cs
order = ?

?0 : Shuffle as bs cs → a , b ∈ as → a , b ∈ cs


## **Exercise 8**

In [104]:
module code.assignment.ex8 where
open import code.shuffle public

len : {A : Set} {as bs cs : A *} →
    Shuffle as bs cs →
    length cs ≡ℕ length as + length bs
len = ?

?0 : Shuffle as bs cs → length cs ≡ℕ length as + length bs


## **Exercise 9**

In [106]:
module code.assignment.ex9 where
open import code.shuffle public

shuffle-map : {A B : Set} {as bs cs : A *} → (f : A → B) →
    Shuffle as bs cs →
    Shuffle (map f as) (map f bs) (map f cs)
shuffle-map = ?

?0
  : (f : A → B) →
    Shuffle as bs cs → Shuffle (map f as) (map f bs) (map f cs)


## **Exercise 10**

In [108]:
module code.assignment.ex10 where
open import code.shuffle public

shuffle-filter : {A : Set} {as bs : A *} (cs : A *) (p : A → 𝔹) →
    Shuffle as bs cs →
    Shuffle (filter p as) (filter p bs) (filter p cs)
shuffle-filter = ?

?0
  : (cs : A *) (p : A → 𝔹) →
    Shuffle as bs cs →
    Shuffle (filter p as) (filter p bs) (filter p cs)
